In [ ]:
#初期設定
import os
import pandas as pd
import numpy as np
import time
import lxml
import re
import requests
from bs4 import BeautifulSoup as bs4
import tqdm
import sys
import inspect
# chromeのwebdriver自動更新用
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [ ]:
# https://web-kiwami.com/python-beautyfulsoup4.html
# http://kondou.com/BS4/
# bs4参考

In [ ]:
# all_year
this_year = 2022 #年変わったら変更
get_year = list(range(2011,this_year+1))
df_result = pd.DataFrame()
dict1 = {}
for year in get_year:
    if 2011 <= year <= 2017:
        print(year, "出力中")
        # 単年のデータ取得2011~2017
        url =  rf"https://www.ipokiso.com/company/{year}.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        time.sleep(np.random.randint(100,120)/100)
        for i in range(len(dfs_list)):
            dfs_list[i]["上場年"] = f"{year}"
            df_result = pd.concat([df_result, dfs_list[i]])
            df_result = df_result.reset_index(drop=True)
        # df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
        # df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
    else:
        # 単年のデータ取得2018~2022
        print(year, "出力中")
        url = rf"https://www.ipokiso.com/company/{year}.html"
        if year == 2022:
            url = r"https://www.ipokiso.com/company/index.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        # サイト更新時にページのtableのデザインが変更していないかチェック
        if year >= 2022:
            try:
                if dfs_list[0].columns[0] != '企業名':
                    raise ValueError("銘柄一覧から取得したtableの値が「企業名」ではありません")
                elif dfs_list[1].columns[0] != '総合評価':
                    raise ValueError("銘柄一覧から取得したtableの値が「総合評価」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")   
        time.sleep(np.random.randint(100,120)/100)
        for i in range(0, len(dfs_list) , 2):
            dfs_list_con = pd.concat([dfs_list[i], dfs_list[i+1]], axis=1)
            dfs_list_con["上場年"] = f"{year}"
            if year == 2019:
                dfs_list_con = dfs_list_con.rename(columns={'上場 市場': '上場市場'})
            if year >= 2020:
                dfs_list_con = dfs_list_con.rename(columns={'申し込み期間': '申し込み 期間', '初値上昇率': '初値 上昇率'})
            df_result = pd.concat([df_result, dfs_list_con],ignore_index=True)
            df_result = df_result.reset_index(drop=True)
        df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
        df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
    if year != this_year:
        time.sleep(np.random.randint(100,120)/100)

In [ ]:
# dict作成
# 空白削除
del dict1[" "]
del dict1[""]
del dict1["\n"]
# dictをdf化
df_dict = pd.DataFrame.from_dict(dict1, orient='index')
# カラム名変更
df_dict = df_dict.rename(columns={0:"url"})
# 間違えているurlを変更
df_dict.url[df_dict.url == "https://www.ipokiso.com/https://www.ipokiso.com/company/2013/zigexn.html"] = "https://www.ipokiso.com/company/2013/zigexn.html"
df_dict = df_dict.rename_axis('index').reset_index()
df_dict = df_dict.rename(columns={'index':'company_name'})

In [ ]:
df_dict

In [ ]:
# 時短のためcode含めたcsv読み込み　コード完成後に削除する
df_result2 = pd.read_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_dict.csv", index_col=0)
df_result2

In [ ]:
# requests.getでのデータ取得用ループ　評価
def make_company_info(url):
    html_res = requests.get(url)
    if html_res.status_code != 200:
        print("requests.getでのurlのアクセスができていません")
        lineno = inspect.currentframe().f_lineno
        print(f"エラーが発生しました。行番号: {lineno}")
        sys.exit()
    time.sleep(np.random.randint(100,120)/100)
    soup = bs4(html_res.content, 'html.parser')
    # 全企業codeの取得
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    code_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        val_list = []
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            val_list.append(keyword)
    except AttributeError:
        val_list = [np.nan,np.nan,np.nan,np.nan]
        print(f"成長性listのエラー{code}")
    evaluation_list.append(val_list)
    print(code)
code_list = []
url_list = df_dict.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:3]] #ここで検証のurl数変更！
df_company_01 = pd.DataFrame(evaluation_list,columns=["成長性","割安性","話題性","総合評価"])
df_result2[df_company_01.columns] = df_company_01
print(df_result2)

ここ以下で作成　12/13～

In [ ]:
#初値予想アンケートの取得実験用　12/13～　これをもとに追加、その後削除する
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
url =  rf"https://www.ipokiso.com/company/2022/beex.html"
driver.get(url)
html_res = driver.page_source.encode('utf-8')
soup = bs4(html_res, 'html.parser')

In [ ]:
# アンケート価格リスト
soup.find_all("p",class_="vote-left")

In [ ]:
# アンケートの票リスト
soup.find_all("div",class_="vote-bar")

In [ ]:
len(soup.find_all("div",class_="vote-bar"))

In [ ]:
vote_values_list = soup.find_all("p",class_="vote-left")
number_of_votes_list = soup.find_all("div",class_="vote-bar")

In [ ]:
votes_list = []
for value in number_of_votes_list:
    # print(value)
    value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
    votes_list.append(value)

In [ ]:
# 20221214編集中 アンケート情報取得する
# seleniumでのデータ取得用ループ　評価
# chromeのwebdriverのinstallしブラウザを起動する 
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
def make_company_info(url):
    driver.get(url)
    #seleniumの場合ここでsleepしないと正しく取得出来ない場合がある
    time.sleep(np.random.randint(100,120)/100)
    # HTMLを文字コードをUTF-8に変換してから取得します。
    html_res = driver.page_source.encode('utf-8')
#ここでシステムエラー処理する？
    # if html_res.status_code != 200:
    #     print("requests.getでのurlのアクセスができていません")
    #     lineno = inspect.currentframe().f_lineno
    #     print(f"エラーが発生しました。行番号: {lineno}")
    #     sys.exit()
    soup = bs4(html_res, 'html.parser')
    # 全企業codeの取得
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    code_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        val_list = []
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            val_list.append(keyword)
    except AttributeError:
        val_list = [np.nan,np.nan,np.nan,np.nan]
        print(f"成長性listのエラー{code}")
    print(code)
    # アンケート調査結果の取得
    print(len(soup.find_all("div",class_="vote-bar")))
    # 6行の場合
    if len(soup.find_all("div",class_="vote-bar")) == 6:
        vote_list = []
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 5行の場合
    elif len(soup.find_all("div",class_="vote-bar")) == 5:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 10行の場合 5行のアンケートがなぜか２つ表示されている銘柄
    elif len(soup.find_all("div",class_="vote-bar")) == 10:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        number_of_votes_list = number_of_votes_list[0:5]
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # アンケートがない場合
    elif len(soup.find_all("div",class_="vote-bar")) == 0:
        val_list.extend([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
    # その他の行数エラー検知
    else:
        print("票の価格帯の数が分岐にありません")
        driver.quit()
        raise Exception("Error: アンケートの価格帯の数がifの分岐にありません")
    # 今まで取得した情報をリストに格納
    evaluation_list.append(val_list)
code_list = []
url_list = df_dict.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:]] #ここで検証のurl数変更！
print(evaluation_list)
df_company_01 = pd.DataFrame(evaluation_list,columns=[
    "成長性",
    "割安性",
    "話題性",
    "総合評価",
    "初値予想_+200%以上",
    "初値予想_+100%以上+200%未満",
    "初値予想_+50%以上+100%未満",
    "初値予想_+20%以上+50%未満",
    "初値予想_+0%以上+20%未満",
    "初値予想_+0%未満",
])
df_result2[df_company_01.columns] = df_company_01
print(df_result2)
driver.quit()

In [ ]:
df_result2.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_result3.csv", encoding='utf-8_sig',index=False)

df_result2

これを元に作成2022/12/09~ ↓　全銘柄チェック、その後、例外の銘柄のtableの変更実施


In [ ]:
# 本番用
def make_company_info_2(url):
    dfs_table_list = pd.read_html(url)
    print(dfs_table_list[1].iloc[0,1])
    time.sleep(np.random.randint(100,120)/100)
    # サイト更新時にデザイン変更がないか検知する（現状ワシントンホテルのみ違うためスルー）
    # 回避urlリスト
    avoidance_url_list = [
        "https://www.ipokiso.com//company/2019/washingtonhotel.html",#[0]に１つ余分に優待情報のtableある
        "https://www.ipokiso.com//company/2020/gmo-fg.html",#table[6][7]が[7][8]にずれている
        "https://www.ipokiso.com//company/2021/frontier.html",#table[6]が無い
        "https://www.ipokiso.com//company/2021/geolocation.html",#table[6]が無い
        ]
    if url not in avoidance_url_list:
        try:
            if dfs_table_list[1].iloc[0,0] != "会社名":
                raise ValueError("銘柄一覧から取得したtableの値が「会社名」ではありません")
            elif dfs_table_list[2].iloc[0,0] != '想定価格':
                raise ValueError("銘柄一覧から取得したtableの値が「想定価格」ではありません")               
            elif dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                raise ValueError("銘柄一覧から取得したtableの値が「抽選申込期間」ではありません")
            elif not '公募' in dfs_table_list[4].iloc[0,0]:
                raise ValueError("銘柄一覧から取得したtableの値が「公募株数等」ではありません")
            elif dfs_table_list[5].iloc[0,1] != '証券会社名':
                raise ValueError("銘柄一覧から取得したtableの値が「証券会社名」ではありません")
        except ValueError as e:
            # エラーが発生した場合の処理
            print("Error: {}".format(e))
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生した行番号: {lineno}")
            print(f"エラーが発生したURL:{url}")
            print(dfs_table_list)
            # sys.exit()
        # 会社名にREITと投資法人という文字列が無いことを判定する
        if not ("REIT" in dfs_table_list[1].iloc[0,1]) and not ("投資法人" in dfs_table_list[1].iloc[0,1]):
            try:
                if dfs_table_list[6].columns[0] != '株主名':
                    raise ValueError("銘柄一覧から取得したtableの値が「株主名」ではありません")
                elif not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    raise ValueError("銘柄一覧から取得したtableの値が「財務データ等」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")
                print(f"エラーが発生したURL:{url}")
                print(dfs_table_list)
                # sys.exit()

    # ここからdfsの結合、整形コード
    df_con_list = []
    for i in range(len(dfs_table_list)):
        # 基本情報
        if i == 1:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if df_table.iloc[0,0] != "会社名":
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：基本情報
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            # 追加する1件分の基本情報のdf作成
            df_table_1 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_1)
            # 表記の揺れ統一のための修正

        # IPO日程と価格決定（初値予想）
        if i == 2:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if dfs_table_list[2].iloc[0,0] != '想定価格':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：IPO日程と価格決定（初値予想）
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_2 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_2)
            # 表記の揺れ統一のための修正
            
        # IPOスケジュール
        if i == 3:
            # tableの位置指定 # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                df_table = dfs_table_list[i + 1]
                print(url)
            
            # df整形：IPOスケジュール
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_3 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_3)
            # 表記の揺れ統一のための修正
        # IPO当選株数
        if i == 4:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if not '公募' in dfs_table_list[4].iloc[0,0]:
                df_table = dfs_table_list[i + 1]
                print(url)
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = dfs_table_list[i + 1]
            
            # df整形：IPO当選株数
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_4 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_4)
            # 表記の揺れ統一のための修正

        # 幹事証券リスト（管理人独自予想あり）
        if i == 5:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[5].iloc[0,1] != '証券会社名':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：幹事証券リスト（管理人独自予想あり）
            df_table_5 = pd.DataFrame()
            for j in range(1,len(df_table.iloc[:,0])):
                val = df_table.iloc[j,:].values
                cols = df_table.iloc[0,:]
                cols[0] = "幹事種類"
                cols = [f"{col}_{j}" for col in cols]
                df_add = pd.DataFrame([val],columns=cols)
                df_table_5 = pd.concat([df_table_5, df_add], axis=1)
            df_con_list.append(df_table_5)

        # 株主構成、ロックアップなど
        if i == 6:
            if dfs_table_list[1].iloc[0,0] == "会社名":
                company_name = dfs_table_list[1].iloc[0,1]
            elif dfs_table_list[2].iloc[0,0] == "会社名":
                company_name = dfs_table_list[2].iloc[0,1]
            if not ("REIT" in company_name or "投資法人" in company_name or  ("福証" in company_name and not "東証" in company_name)):
                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if dfs_table_list[6].columns[0] != '株主名':
                    df_table = dfs_table_list[i + 1]
                    print(url)
                df_table_6 = pd.DataFrame()
                for k in range(0,len(df_table.iloc[:,0])):
                    val = df_table.iloc[k,:].values
                    cols = df_table.columns
                    cols = [f"{col}_{k+1}" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_6 = pd.concat([df_table_6, df], axis=1)
                df_con_list.append(df_table_6)

            # 企業業績のデータ（5年分）
        if i == 7:
            if not ("REIT" in company_name or "投資法人" in company_name):

                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    df_table = dfs_table_list[i + 1]
                    print(url)
                
                df_table_7 = pd.DataFrame()
                for h in range(1,len(df_table.T.iloc[:,0])):
                    val = df_table.T.iloc[h,:].values
                    # カラム名の誤字修正
                    cols = df_table.T.iloc[0,:]
                    cols[0] = "年月"
                    cols = [f"{col}_{h}年目" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_7 = pd.concat([df_table_7, df], axis=1)
                df_con_list.append(df_table_7)


            # 全件のdfに追加していく
    df_table_all = pd.concat(df_con_list,axis=1)
    print(url)
    print("finish")
    return df_table_all
df_all = pd.DataFrame()
url_list = df_dict.url.values
for url in url_list[0:]:#ここでurl数変更
    df_table_all = make_company_info_2(url) 
    df_all = pd.concat([df_all,df_table_all],axis=0,ignore_index=True)
print(df_all)

In [ ]:
df_all

In [ ]:
df_all.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_all.csv", encoding='utf-8_sig',index=False)

In [ ]:
df_all_copy = pd.read_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_all.csv")


以上本番用

In [ ]:
""" 
# csv変換 時短のためこれを読み込む
df_code = pd.DataFrame(code_list,columns=["code"])
df_dict[df_code.columns] = df_code.values
df_result2.to_csv("df_result2", encoding='utf-8_sig')
"""

In [ ]:
# 時短のためcode含めたcsv読み込み
df_result2 = pd.read_csv("df_dict.csv", index_col=0)
df_result2

- IPO当選株数のデータ取得

In [ ]:
# 以下データ成長性等取得する
url = "https://www.ipokiso.com//company/2019/washingtonhotel.html"
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')

In [ ]:
dfs_table_list = pd.read_html("https://www.ipokiso.com//company/2016/ichigo_green.html")

In [ ]:
"REIT" in dfs_table_list[1].iloc[0,1]
"投資法人" in dfs_table_list[1].iloc[0,1]

In [ ]:
if ("REIT" in dfs_table_list[1].iloc[0,1 ] or "投資法人" in dfs_table_list[1].iloc[0,1]):
    print("hi")


In [ ]:
df_html_table_2011[7]


In [ ]:
name

In [ ]:
df_html_table = pd.read_html("https://www.ipokiso.com/company/2022/tms-japan.html")
df_html_table

[0]の取得

In [ ]:
df_company_01 = pd.DataFrame(evaluation_list,columns=["成長性","割安性","話題性","総合評価"])
df_result2[df_company_01.columns] = df_company_01
print(df_result2)

In [ ]:
# code_listの作成（詳細情報の取得ループ 全てのcode取得[0]までok！
def make_company_info(url):
    html_res = requests.get(url)
    if html_res.status_code != 200:
        print("requests.getでのurlのアクセスができていません")
        lineno = inspect.currentframe().f_lineno
        print(f"エラーが発生しました。行番号: {lineno}")
        sys.exit()
    time.sleep(np.random.randint(100,120)/100)
    soup = bs4(html_res.content, 'html.parser')
    # 全企業codeの取得
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    code_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        val_list = []
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            else:
                keyword = np.nan
            val_list.append(keyword)
    except AttributeError:
        val_list = [np.nan,np.nan,np.nan,np.nan]
        print(f"成長性listのエラー{code}")
    evaluation_list.append(val_list)
    print(code)
code_list = []
url_list = df_dict.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:3]] #ここで検証のurl数変更！
df_company_01 = pd.DataFrame(evaluation_list,columns=["成長性","割安性","話題性","総合評価"])
df_result2[df_company_01.columns] = df_company_01
print(df_result2)


In [ ]:
df_result2

In [ ]:
df_all.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_all.csv", encoding='utf-8_sig')

全table取得（pd_htmlによる）

In [ ]:
df_html_table[2].T

In [ ]:
df_table_1

In [ ]:
cols = df_html_table[2].T.values[0]
# 基本情報のval取得
val = df_html_table[2].T.values[1]
# 追加する1件分の基本情報のdf作成
df_table_1 = pd.DataFrame([val],columns=cols)
df_table_1

In [ ]:
cols = df_html_table[3].T.values[0]
# 基本情報のval取得
val = df_html_table[3].T.values[1]
# 追加する1件分の基本情報のdf作成
df_table_2 = pd.DataFrame([val],columns=cols)
df_table_2

In [ ]:
cols = df_html_table[4].T.values[0]
# 基本情報のval取得
val = df_html_table[4].T.values[1]
# 追加する1件分の基本情報のdf作成
df_table_3 = pd.DataFrame([val],columns=cols)
df_table_3

In [ ]:
val[5]

In [ ]:
df_html_table[5].T.values[2]

In [ ]:
cols = df_html_table[5].T.values[0]
# 基本情報のval取得
val = df_html_table[5].T.values[1]
# 追加する1件分の基本情報のdf作成
df_table_4 = pd.DataFrame([val],columns=cols)
df_table_4

In [ ]:
pd.concat([df_table_1,df_table_2],axis=0)

In [ ]:
df_all = pd.DataFrame()

In [ ]:
if df_all.empty == True:
    print("hi")

In [ ]:
del df_all

In [ ]:
df_all = pd.concat([df_all,df_table_all],axis=0)

In [ ]:
df_table_all = pd.merge(df_table_1,df_table_2,on=["key"])

In [ ]:
# 20221201 ループで全table、全銘柄取り出す。（細かいデータの整形は後でやる）
def make_company_info(url):
    df_html_table = pd.read_html(url)
    for i in range(0,8):

        # 基本情報
        if i == 0:
            df_table = df_html_table[i + 1]
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = df_html_table[i + 2]
            # 基本情報のcols取得
            cols = df_table.T.values[0]
            # 基本情報のval取得
            val = df_table.T.values[1]
            # 追加する1件分の基本情報のdf作成
            df_table_1 = pd.DataFrame([val],columns=cols)
        # IPO日程と価格決定（初値予想）
        if i == 1:
            df_table = df_html_table[i + 1]
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = df_html_table[i + 2]
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_2 = pd.DataFrame([val],columns=cols)
        # 次ここから
        # if i == 2:

        # 全件のdfに追加していく
    df_table_all = pd.concat([df_table_1,df_table_2],axis=1)
    
    return df_table_all
df_all = pd.DataFrame()
code_list = []
url_list = df_dict.url.values
evaluation_list = []
for url in url_list[0:3]:
    df_table_all = make_company_info(url)
    df_all = pd.concat([df_all,df_table_all],axis=0)
# df = [make_company_info(url) for url in url_list[0:3]] #ここで検証のurl数変更！


[1]の取得

In [ ]:
df_html_table_2011 = pd.read_html("https://www.ipokiso.com//company/2019/washingtonhotel.html")
cols = df_html_table_2011.T.values[0]
val = df_html_table_2011.T.values[1]
df_basic_info2 = pd.DataFrame([val],columns=cols)
df_basic_info2

In [ ]:
df_html_table_2011 = pd.read_html("https://www.ipokiso.com//company/2019/washingtonhotel.html")[1]
cols = df_html_table_2011.T.values[0]
val = df_html_table_2011.T.values[1]
df_basic_info = pd.DataFrame([val],columns=cols)
df_basic_info

In [ ]:
df_html_table_2011 = pd.read_html("https://www.ipokiso.com//company/2019/washingtonhotel.html")

df_html_table_2011[2]

In [ ]:
# 時短のためcode含めたcsv読み込み
df_result2 = pd.read_csv("df_dict.csv", index_col=0)
df_result2

In [ ]:
# 202221130
def make_company_info(url):
    html_res = requests.get(url)
    if html_res.status_code != 200:
        print("requests.getでのurlのアクセスができていません")
        lineno = inspect.currentframe().f_lineno
        print(f"エラーが発生しました。行番号: {lineno}")
        sys.exit()
    time.sleep(np.random.randint(100,120)/100)
    soup = bs4(html_res.content, 'html.parser')
    # 全企業codeの取得
    try:
        code = re.search(r"[0-9]{4}", soup.title.text).group()
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = re.search(r"[0-9]{4}", f).group()
    code_list.append(code)
    # table[0]成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        val_list = []
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            else:
                keyword = np.nan
            val_list.append(keyword)
    except AttributeError:
        val_list = [np.nan,np.nan,np.nan,np.nan]
        print(f"成長性listのエラー{code}")
    evaluation_list.append(val_list)

    # read_htmlでtable情報取得
    df_html_table = pd.read_html(url)

    #[1]基本情報
    # 関数外の変数が取れないためglobalで定義する
    global df_basic_info
    time.sleep(np.random.randint(100,120)/100)
    for i in range(1,8):
        df_table = df_html_table[1 + i].copy()
        # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
        if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
            df_table = df_html_table[2 + i].copy()
        # 基本情報のcols取得
        cols = df_table.T.values[0]
        # 基本情報のval取得
        val = df_table.T.values[1]
        # 追加する1件分の基本情報のdf作成
        df_table_new = pd.DataFrame([val],columns=cols)
        print(df_table_new)
        # 全件のdfに追加していく
        # df_basic_info = pd.merge([df_basic_info,df_table_new])
        return df_basic_info

    """
    # table基本情報のみ取取得
    df_table_basic = df_html_table[1].copy()
    # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
    if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
        df_table_basic = df_html_table[2].copy()
    # 基本情報のcols取得
    cols = df_table_basic.T.values[0]
    # 基本情報のval取得
    val = df_table_basic.T.values[1]
    # 追加する1件分の基本情報のdf作成
    df_basic_info_new = pd.DataFrame([val],columns=cols)
    # 全件のdfに追加していく
    df_basic_info = pd.concat([df_basic_info,df_basic_info_new],ignore_index=True)
    print(code)
    print(evaluation_list)
    return df_basic_info
    """
    
code_list = []
url_list = df_dict.url.values
evaluation_list = []


# 基本情報の空df作成
df_basic_info = pd.DataFrame([],columns=["会社名","コード","市場","会社URL","狙い目証券会社","会社設立"])
[make_company_info(url) for url in url_list[range(3)]] #ここで検証のurl数変更！
df_company_01 = pd.DataFrame(evaluation_list,columns=["成長性","割安性","話題性","総合評価"])
df_result2[df_company_01.columns] = df_company_01
#[1]データ分割、整理、結合　codeの場所を分ける？ 
# 結合(axis=1で横方向の結合)
df_result2 = pd.concat([df_result2,df_basic_info],axis=1)


print(df_result2)

In [ ]:
df_result2

In [ ]:
df_basic_info.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_basic_info.csv", encoding='utf-8_sig')

[2]の取得

[3]の取得

[4]の取得

[5]の取得

[6]の取得

[7]の取得

以降参考データ

In [ ]:
df_html_table.T.iloc[1:,:].values

In [ ]:
df_html_table = pd.read_html("https://www.ipokiso.com/company/2022/tms-japan.html")[4]

cols = df_html_table.T.iloc[0,:].values
val = df_html_table.T.iloc[1:,:].values.flatten()
df_ipo_stock_info = pd.DataFrame([val],columns=cols)


In [ ]:
val

In [ ]:
# re.subで文字列置換している　(arr.split("株")[i])の文字列のr'/D'に一致したものを''に置換する
# \Dは数字以外の文字列
# splitで株で区切って株自体は消す。配列にいれる
# intで数字に変換
stock_info = [
    int(re.sub( r'\D', '', (arr.split("株")[i]))) 
    for arr in val
    for i in range(3) 
]
# ravelは配列を一次元化する
# ravel()は可能な限りビューを返すが、flatten()は常にコピーを返すという違いがある。
# カラムを２つ国内と海外を作成してリスト化、1次元化している。
cols_1 = np.ravel([[c,c+"_国内",c+"_海外"] for c in cols])
# 分割したデータと、名称を変更したカラムでデータフレームを作成している。
df_stock_info = pd.DataFrame([stock_info],columns=cols_1)

以後チェック用code

In [ ]:
# URLdict（単年）作成 確認用
year = "2020"
url = f'https://www.ipokiso.com/company/{year}.html'
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')
find_all_list = soup.find_all(href=re.compile(f"company/{year}/"))
url_dict = {
    find_all_list[i].text:"https://www.ipokiso.com/" + soup.find_all(href=re.compile(f"company/{year}/"))[i]["href"]
    for i in range(len(find_all_list))
}
url_dict

In [ ]:
# 単年のデータ取得2011~2017　確認用
df_result = pd.DataFrame()
year = "2011"
url =  rf"https://www.ipokiso.com/company/{year}.html"
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')
find_all_list = soup.find_all(href=re.compile(f"company/{year}/"))
print(len(find_all_list))
df_url = pd.read_html(url)
time.sleep(np.random.randint(100,120)/100)
for i in range(len(df_url)):
    df_url[i]["上場年"] = f"{year}"
    df_result = pd.concat([df_result, df_url[i]])
    df_result = df_result.reset_index(drop=True)
df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
df_result

In [ ]:
# 単年のデータ取得2018~2022　確認用
df_result = pd.DataFrame()
year = "2022"
url =  rf"https://www.ipokiso.com/company/{year}.html"
if year == "2022":
    url =  r"https://www.ipokiso.com/company/index.html"
html_res = requests.get(url)
time.sleep(np.random.randint(100,120)/100)
soup = bs4(html_res.content, 'html.parser')
find_all_list = soup.find_all(href=re.compile(f"company/{year}/"))
print(len(find_all_list))
df_url = pd.read_html(url)
time.sleep(np.random.randint(100,120)/100)
for i in range(0, len(df_url) , 2):
    df_url_con = pd.concat([df_url[i], df_url[i+1]], axis=1)
    df_url_con["上場年"] = f"{year}"
    df_result = pd.concat([df_result, df_url_con],ignore_index=True)
    df_result = df_result.reset_index(drop=True)
df_result.drop(df_result[df_result['企業名'] == "企業名"].index , inplace=True)
df_result.drop(df_result[df_result['初値'] == "初値"].index , inplace=True)
df_result

In [ ]:
"""ArithmeticErrordf = df.drop_duplicates()
#カラムの重複した行を消す。
df = df[(df['申込日'] != "申込日").values].reset_index(drop=1)
#index番号をリセットする。
col = ['申込日', '株数', '前週比', '金額', '前週比.1', '株数.1', '前週比.1.1', '金額.1', '前週比.1.2','評価損益率(%)', '信用倍率']
df.columns = col
#columns（カラムズ）パラメータで列名を自分で指定し修正する。
#2重になってたカラムも一つに統一できる。MultiIndex⇒Indexにパラメータを変更。
"""

In [ ]:
# df.to_csv(r"C:\Users\xxp2p\OneDrive\ドキュメント\MEGA_saya\Traders_web\margin_transition\csv\margin_transition.csv", encoding='utf-8_sig')
#文字化け改善 encoding='utf-8_sig

In [ ]:
#読み込み
# pd.read_csv(r"C:\Users\xxp2p\OneDrive\ドキュメント\MEGA_saya\Traders_web\margin_transition\csv\margin_transition.csv", header=None, na_values=['-'])

In [ ]:
#Pythonの文字列を拡張するf,r,b,uについて
#f… .format()を使うことなく文字列の中に変数を埋め込むことができる。
#r… raw string　通常はバックスラッシュ（改行）があった場合エスケープシーケンスが働くが、raw stringを用いることで無視。
#b…バイト列リテラル str型ではなくバイト型のインスタンスを作成します。
#u…Unicodeに変換する
#fとrだけ使えればOK！

In [ ]:
#縦方向に結合する方法|append(), concat()  参考URL https://obgynai.com/python-pandas-index-merge-join/
#append：新しい行を追加するメソッド  
#concat：columns(列名)や index (行名)を参照して結合するメソッド。ignore_indexとはindexを無視するという意味。concatでデータフレームを結合した際にindexに付与された番号を無視すると言う事になる。
#drop dropで行を削除する  参考URL　https://www.sejuku.net/blog/73830

In [ ]:
#データのスクレイピングとデータ整形のコードは分ける。